In [6]:
from __future__ import unicode_literals, print_function, division
import os
import glob

from io import open
from pprint import pprint

In [7]:
def find_files(path):
    return glob.glob(path)

pprint(find_files('./data/names/*.txt'))

['./data/names/German.txt',
 './data/names/Vietnamese.txt',
 './data/names/Italian.txt',
 './data/names/Dutch.txt',
 './data/names/Polish.txt',
 './data/names/Korean.txt',
 './data/names/Japanese.txt',
 './data/names/Irish.txt',
 './data/names/French.txt',
 './data/names/Spanish.txt',
 './data/names/English.txt',
 './data/names/Greek.txt',
 './data/names/Portuguese.txt',
 './data/names/Chinese.txt',
 './data/names/Russian.txt',
 './data/names/Czech.txt',
 './data/names/Arabic.txt',
 './data/names/Scottish.txt']


In [8]:
import string
import unicodedata

In [15]:
all_letters = string.ascii_letters + ".,;'"
n_letters = len(all_letters)

In [23]:
unicodedata.category('')

'Po'

In [24]:
# 유니코드 문자열을 ASCII로 변환, https://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [26]:
print(unicode_to_ascii('Ślusàrski'))

Slusarski


In [37]:
# 각 언어의 이름 목록인 category_lines 사전 생성
category_lines = {}
all_categories = []

def read_lines(filename):
    """ 파일을 읽고 줄 단위로 분리 """
    lines = open(filename, encoding='utf8').read().strip().split('\n')
    return [unicode_to_ascii(line) for line in lines]


for filename in find_files('./data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = read_lines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)
print("Number of categories: ", n_categories)
print("Some category examples: ")
pprint(all_categories[:3])
print("One category <{}> examples: ".format(all_categories[0]))
pprint(category_lines[all_categories[0]][:3])

Number of categories:  18
Some category examples: 
['German', 'Vietnamese', 'Italian']
One category <German> examples: 
['Abbing', 'Abel', 'Abeln']


###  이름을 Tensor로 변경

모든 이름을 담았으니, 이를 활용하기 위해 Tensor로 변환.

하나의 문자를 표현하기 위해, 크기가 `1 x n_letters` 인 one-hot 벡터를 사용.

one-hot 벡터는 현재 문자의 주소에만 1을 값으로 갖고 그외엔 0으로 채워짐.

단어를 만들기 위해, one-hot 벡터들을 2차원 matrix `word_length x 1 x n_letters` 로 표현한다

위의 추가된 1차원은 PyTorch에서 모든 것이 Batch에 있다 가정하기때문에 발생함. 여기선 배치크기 1을 사용.

In [39]:
'''
One-hot 벡터는 언어를 다룰 때 자주 사용하며
단어, 글자 등을 벡터로 표현할때 단어, 글자 사이의 상관관계를 미리 알 수 없을 경우에 
One-hot 으로 표현하여 서로 직교한다 가정하고 학습을 시작함.
동일하게 상관 관계를 알 수 없는 다른 데이터의 경우에도 One-hot 벡터를 활용가능합니다.
'''

import torch


In [41]:
# all_letters 로 문자 index 찾기, 예시 "a" = 0 
def letter_to_index(letter):
    return all_letters.find(letter)

# 검증을 위해 한개의 문자를 <1xn_letters> Tensor로 변환
def letter_to_tensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letter_to_index(letter)] = 1
    return tensor

# 한 줄(이름 단어)을 <word_length x 1 x n_letters>,
# 혹은 one-hot 문자 벡터의 Array로 변경
def line_to_tensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letter_to_index(letter)] = 1
    return tensor

In [44]:
print(line_to_tensor('Jones').size())

torch.Size([5, 1, 56])


In [48]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size,  output_size):
        super().__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input_, hidden):
        combined = torch.cat((input_, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

In [49]:
n_hidden = 128
rnn = RNN(
    input_size = n_letters,
    hidden_size = n_hidden,
    output_size = n_categories
)


In [54]:
input_ = letter_to_tensor('A')

In [55]:
hidden = torch.zeros(1, n_hidden)

In [56]:
output, next_hidden = rnn(input_, hidden)
print("Output: ", output)
print("Next hidden: ", next_hidden)

Output:  tensor([[-2.8766, -2.9238, -2.8288, -2.8049, -2.8781, -2.9420, -2.8577, -2.9947,
         -2.9694, -2.8341, -2.9029, -2.8907, -2.8281, -2.8761, -2.9581, -2.9162,
         -2.8345, -2.9353]], grad_fn=<LogSoftmaxBackward>)
Next hidden:  tensor([[ 2.9812e-02,  9.0886e-02,  2.4523e-02, -1.2931e-02, -4.4243e-02,
         -4.0083e-02, -9.4140e-02, -3.3278e-02,  5.9341e-03, -4.3118e-02,
          1.0369e-01,  5.9213e-02, -8.6161e-02,  6.4016e-02,  4.3286e-02,
          6.4705e-02, -1.9361e-02, -5.6345e-02, -2.5444e-02, -8.7744e-02,
          3.2230e-03,  1.2015e-01, -4.7014e-02,  9.6912e-02, -3.7209e-02,
          7.9992e-02,  3.5023e-02, -7.7128e-02,  9.1777e-03, -1.2559e-02,
         -2.2583e-03,  3.8386e-02, -5.9560e-02, -4.9146e-02, -1.1979e-01,
          6.6761e-02,  5.0746e-02,  3.5824e-02, -6.0577e-02,  4.1560e-02,
          4.5911e-02, -9.9856e-03,  4.8818e-02,  2.5867e-02, -5.1356e-02,
         -1.0025e-01,  7.5032e-02, -5.5703e-03, -6.9703e-02,  4.2210e-02,
         -5.5812

In [57]:
input_ = line_to_tensor('Albert')
hidden = torch.zeros(1, n_hidden)
output, next_hidden = rnn(input_[0], hidden)
print(output)

tensor([[-2.8766, -2.9238, -2.8288, -2.8049, -2.8781, -2.9420, -2.8577, -2.9947,
         -2.9694, -2.8341, -2.9029, -2.8907, -2.8281, -2.8761, -2.9581, -2.9162,
         -2.8345, -2.9353]], grad_fn=<LogSoftmaxBackward>)


### Training

학습에 들어가기전에 몇몇 도움되는 함수를 만들어야 합니다. 첫째는 우리가 알아낸 

각 카테고리의 우도인 네트워크 출력을 해석하는 것 입니다. 가장 큰 값의 주소를 알아내기 위해서 `Tensor.topk` 함수를 사용가능합니다. 

즉, 네트워크 출력(각 카테고리의 우도) 으로 가장 확률이 높은 카테고리 이름(언어)과 카테고리 번호 반환

In [59]:
def category_from_output(output):
    top_n, top_i = output.topk(1) # 텐서의 가장 큰 값과 주소
    category_i = top_i[0].item() # 텐서에서 정수값으로 변경
    return all_categories[category_i], category_i

In [60]:
print(category_from_output(output))

('Dutch', 3)
